In [2]:
import duckdb
import os
from dotenv import load_dotenv
import pandas as pd
import matplotlib.pyplot as plt
import geopandas as gpd
from shapely.geometry import Point
load_dotenv()

# Recuperar as variáveis de ambiente
hmac_access = os.getenv('HMAC_ACCESS_KEY_ID')
hmac_secret = os.getenv('HMAC_SECRET_ACCESS_KEY')

# Conectar ao DuckDB
con = duckdb.connect(database='telemetria.duckdb')


duckdb.sql("LOAD httpfs;")
duckdb.sql(f"SET s3_access_key_id='{hmac_access}';")
duckdb.sql(f"SET s3_secret_access_key='{hmac_secret}';")
duckdb.sql("SET s3_endpoint='storage.googleapis.com'")
results = duckdb.sql("SELECT * FROM read_parquet('gs://hp-operacao-raw/mix/geodata/geodata_*.parquet', union_by_name=True)").df()


In [3]:
# Salvar o DataFrame em uma nova tabela no DuckDB
con.execute("CREATE TABLE geodata AS SELECT * FROM results")

# Verificar se a tabela foi criada corretamente
# con.execute("SELECT * FROM positions LIMIT 10").df()
con.close()


In [4]:
con.close()

In [6]:
con = duckdb.connect(database='telemetria.duckdb')

In [1]:
import duckdb
import os
from dotenv import load_dotenv

# Carregar as variáveis de ambiente
load_dotenv()
hmac_access = os.getenv('HMAC_ACCESS_KEY_ID')
hmac_secret = os.getenv('HMAC_SECRET_ACCESS_KEY')

# Conectar ao DuckDB
con = duckdb.connect(database='telemetria.duckdb')

# Configurar DuckDB para acessar o Google Cloud Storage
duckdb.sql("LOAD httpfs;")
duckdb.sql(f"SET s3_access_key_id='{hmac_access}';")
duckdb.sql(f"SET s3_secret_access_key='{hmac_secret}';")
duckdb.sql("SET s3_endpoint='storage.googleapis.com'")

# Função para criar tabela a partir dos arquivos Parquet em uma subpasta
def create_table_from_parquet(subfolder):
    # Nome da tabela baseado no nome da subpasta
    table_name = subfolder.replace("gs://hp-operacao-raw/mix/", "").replace("/", "_")
    
    try:
        # Obter metadados para verificar a estrutura dos arquivos
        metadata_query = f"SELECT * FROM parquet_metadata('{subfolder}*.parquet')"
        metadata_df = duckdb.sql(metadata_query).df()
        
        # Verificar a consistência dos arquivos Parquet
        # (Esta parte pode ser personalizada conforme a necessidade para verificar a consistência)
        
        # Ler os arquivos Parquet da subpasta
        query = f"SELECT * FROM read_parquet('{subfolder}*.parquet', union_by_name=True)"
        df = duckdb.sql(query).df()
        
        # Criar tabela no DuckDB
        con.execute(f"CREATE TABLE {table_name} AS SELECT * FROM df")
        print(f"Tabela '{table_name}' criada com sucesso.")
    except Exception as e:
        print(f"Erro ao criar a tabela '{table_name}': {e}")

subfolders = [
    # 'gs://hp-operacao-raw/mix/events/',
    'gs://hp-operacao-raw/mix/geodata/',
    # 'gs://hp-operacao-raw/mix/location/',
    # 'gs://hp-operacao-raw/mix/positions/'
]

# Criar tabelas para cada subpasta
for subfolder in subfolders:
    create_table_from_parquet(subfolder)

# Fechar a conexão
con.close()


In [4]:
con.close()

In [ ]:
subfolders = [
    # 'gs://hp-operacao-raw/mix/positions/',
    # 'gs://hp-operacao-raw/mix/cars/',
    # 'gs://hp-operacao-raw/mix/dict_events/',
    'gs://hp-operacao-raw/mix/events/',
    # 'gs://hp-operacao-raw/mix/geodata/',
    # 'gs://hp-operacao-raw/mix/location/',
    
]